## Import libraries


In [1]:
import pandas as pd
import numpy as np
import os
import json

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from bayes_opt import BayesianOptimization
from sklearn.utils import shuffle
from catboost import CatBoostClassifier


## Data loading

In [2]:
file_path  = os.path.join(os.getcwd(), '../data', 'processed_data.tsv')
df = pd.read_csv(file_path, sep='\t')
df

,target,0,1,2,3,4,5,6,7,8,...,189,190,191,192,193,194,195,196,197,198
0,SERVICE,-0.128368,-0.469331,-0.177878,-0.192006,-0.222916,-0.093777,0.025445,-0.832899,-0.436870,...,-0.062861,-0.510515,-0.763284,-0.378731,0.528817,-0.049521,-0.329912,0.620485,-0.031838,-0.507967
1,SERVICE,-0.163475,-0.396297,-0.157724,-0.146050,-0.182638,-0.081966,0.078536,-0.641592,-0.299122,...,-0.295817,0.414417,-0.198974,0.003079,0.043172,-0.196704,-0.235280,0.157981,-0.224459,1.148332
2,NON_FOOD_GOODS,-0.327237,-0.892746,-0.421341,-0.114922,0.145603,0.178709,-0.678991,0.014879,-0.191826,...,-2.362266,-2.083801,4.348409,2.111854,8.086853,0.882947,1.015955,0.833849,1.022884,-0.328521
3,LOAN,-0.282064,-0.650740,-0.145728,-0.065063,-0.387599,-0.455714,0.259525,-1.033327,-0.479710,...,-0.021922,0.351193,-0.078121,-0.190551,0.202207,-0.030612,-0.136432,0.240200,0.016642,-0.120984
4,NOT_CLASSIFIED,-0.494004,-1.070177,-0.595625,-0.433807,-0.470061,-0.099003,0.713364,-0.616545,-0.117873,...,-0.618260,0.101671,-0.824624,-0.097199,0.223601,-0.150128,-0.369099,1.199207,0.386687,-0.350164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,TAX,-0.265042,-0.515122,-0.227587,-0.170178,-0.028201,0.028267,-0.192035,-0.286995,-0.204795,...,0.400836,-0.280797,0.010601,0.159663,-0.035141,-0.065599,-0.141998,0.008215,-0.165131,0.121020
496,BANK_SERVICE,-2.048056,20.911902,-10.326095,-3.513630,0.001023,-0.810541,-0.704414,0.892176,-4.624454,...,0.003252,0.008277,0.006612,-0.018563,-0.003845,-0.003299,-0.004852,0.004102,0.007205,0.026897
497,SERVICE,-0.131671,-0.358712,-0.142820,-0.118663,-0.166825,-0.083802,0.085220,-0.600441,-0.291151,...,-0.249828,1.571986,0.073639,-0.104947,0.662228,-0.267097,-1.113358,-0.172560,0.554598,-0.264534
498,FOOD_GOODS,-0.624792,-1.234122,-0.523791,-0.253499,-0.185988,-0.144191,0.002712,-0.402693,0.017636,...,-0.112007,-0.201682,0.092288,0.080106,-0.013846,0.029819,0.083273,-0.144005,-0.165861,-0.013142


In [3]:
X = df.drop(columns=['target'])
y = df['target']

X, y = shuffle(X, y, random_state=42)


## Model

In [4]:
def catboost_cv(learning_rate, depth, iterations, min_data_in_leaf, max_bin):
    model = CatBoostClassifier(
        learning_rate=learning_rate,
        depth=int(depth),
        iterations=int(iterations),
        min_data_in_leaf=int(min_data_in_leaf),
        max_bin=int(max_bin),
        cat_features=[],  # Укажите индексы категориальных признаков, если таковые имеются
        thread_count=-1,
        random_seed=42,
        verbose=0
    )
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    
    for train_idx, val_idx in kf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        accuracies.append(accuracy_score(y_val, y_pred))
        precisions.append(precision_score(y_val, y_pred, average='macro', zero_division=0))
        recalls.append(recall_score(y_val, y_pred, average='macro'))
        f1_scores.append(f1_score(y_val, y_pred, average='macro'))
    
    return np.mean(accuracies)


## Selection of hyperparameters

In [5]:
param_bounds = {
    'learning_rate': (0.07, 0.095), # Скорость обучения
    'depth': (3, 6),  # Глубина дерева
    'iterations': (100, 200),  # Количество итераций
    'min_data_in_leaf': (30, 40), # Минимальное количество данных в листе
    'max_bin': (100, 200) # Максимальное количество бинов для гистограммы
}

optimizer = BayesianOptimization(
    f=catboost_cv,
    pbounds=param_bounds,
    random_state=42
)

optimizer.maximize(
    init_points=5,  
    n_iter=10,   
)

print("Best hyperparameters found: ", optimizer.max)


|   iter    |  target   |   depth   | iterat... | learni... |  max_bin  | min_da... |
-------------------------------------------------------------------------------------
| 1         | 0.94      | 4.124     | 195.1     | 0.0883    | 159.9     | 31.56     |
| 2         | 0.908     | 3.468     | 105.8     | 0.09165   | 160.1     | 37.08     |
| 3         | 0.944     | 3.062     | 197.0     | 0.09081   | 121.2     | 31.82     |
| 4         | 0.92      | 3.55      | 130.4     | 0.08312   | 143.2     | 32.91     |
| 5         | 0.926     | 4.836     | 113.9     | 0.0773    | 136.6     | 34.56     |
| 6         | 0.94      | 3.889     | 172.6     | 0.07575   | 100.1     | 37.15     |
| 7         | 0.942     | 4.352     | 197.7     | 0.09069   | 121.6     | 32.8      |
| 8         | 0.94      | 3.0       | 176.4     | 0.07      | 128.9     | 30.0      |
| 9         | 0.94      | 3.0       | 198.9     | 0.07      | 100.0     | 30.0      |
| 10        | 0.932     | 4.469     | 199.0     | 0.07

## Best model

In [6]:
best_params = optimizer.max['params']

learning_rate_best = best_params['learning_rate']
depth_best = int(best_params['depth'])
iterations_best = int(best_params['iterations'])
min_data_in_leaf_best = int(best_params['min_data_in_leaf'])
max_bin_best = int(best_params['max_bin'])

best_model = CatBoostClassifier(
    learning_rate=learning_rate_best,
    depth=depth_best,
    iterations=iterations_best,
    min_data_in_leaf=min_data_in_leaf_best,
    max_bin=max_bin_best,
    cat_features=[],  
    thread_count=-1,
    random_seed=42,
    verbose=0
)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

accuracies = []
precisions = []
recalls = []
f1_scores = []

for train_idx, val_idx in kf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_val)
    
    accuracies.append(accuracy_score(y_val, y_pred))
    precisions.append(precision_score(y_val, y_pred, average='macro', zero_division=0))
    recalls.append(recall_score(y_val, y_pred, average='macro'))
    f1_scores.append(f1_score(y_val, y_pred, average='macro'))

print("Mean Accuracy: ", round(np.mean(accuracies), 3))
print("Mean Precision: ", round(np.mean(precisions), 3))
print("Mean Recall: ", round(np.mean(recalls), 3))
print("Mean F1-Score: ", round(np.mean(f1_scores), 3))


Mean Accuracy:  0.922
Mean Precision:  0.951
Mean Recall:  0.923
Mean F1-Score:  0.928


## Saving parameters

In [ ]:
best_params = optimizer.max['params']
output_file_path  = os.path.join(os.getcwd(), '../best_parameters', 'catboost_params.json')

with open(output_file_path, "w") as file:
    json.dump(best_params, file, indent=4)  

print(f"Best parameters saved to {output_file_path}")
